When computing the position in the orbit we have to solve the transcendental equation

$$ M = E - e \sin E $$

Write it down as Danby(1988)

$$ f(E) = E - e \sin E - M $$

and use the Newton-Raphson iteration method to find the root of the nonlinear equation $f(E)=0$. The iteration scheme

$$ E_{i + 1} = E_i - \frac{f(E_i)}{f'(E_i)} $$

with $i = 0,1,2,...,$ and $f'(E_i) = \mathrm{d}f(E_i)/\mathrm{d}E_i = 1 - e \mathrm{cos} E_i.$

Substituting $f(E)$ and $f'(E)$ into the iteration scheme $E_{i+1}$ we obtain 

$$E_{i+1} = E_{i} - \frac{E_{i} - e \sin E_{i} - M}{1 - e \cos E_{i}} $$

Initial value $E_{0}$

Danby (1988) points out that, by first reducing $M$ to the range
$0 \leq M \leq 2 \pi$, the initial guess

$$E_0 = M + \mathrm{sign}(\sin M)k e $$

with $0 \leq k \leq 1$ and we will make use of the recomended value $k=0.85$. Further details of this and other methods are discussed
in Danby & Burkardt (1983), Burkardt & Danby (1983), and Danby (1987).

In [1]:
import math 
import numpy as np
import random

In [2]:
mean_anom = (2.*np.pi)*random.random() #mean_anomaly



In [47]:
ecc_bin = 0.9
kk = 0.85
E0 = mean_anom + np.sign(np.sin(mean_anom))*kk*ecc_bin

In [48]:
print(E0)

0.9600727261440201


In [49]:
E1 = E0 - (E0 - ecc_bin*np.sin(E0) - mean_anom)/(1 - ecc_bin*np.cos(E0))

In [50]:
print(E1)

0.9028483539163896


In [51]:
ifor = 8
Ei1 = E0 #Define E_{i-1}
for i in range(ifor):
    Ei = Ei1 - (Ei1 - ecc_bin*np.sin(Ei1) - mean_anom)/(1 - ecc_bin*np.cos(Ei1))   
    Ei1 = Ei
    print(Ei)

0.9028483539163896
0.9001577384940253
0.9001519385461373
0.900151938519225
0.9001519385192249
0.9001519385192248
0.9001519385192247
0.9001519385192248


In [30]:
print(Ei)

1.0726768003470104


In [13]:
ecc_anom = Ei


In [14]:
#------------------------------------------------------------------------------------------
#Units and conversions:
#------------------------------------------------------------------------------------------
#code units: Rsun, Msun, G=1, ...
c_SI       = 299792458.0        #m/s
M_sun_SI   = 1.989*(10.**30.)   #kg
R_sun_SI   = 695800000.         #m
AU_SI      = 149597871000.      #m 
G_new_SI   = 6.67*(10.**(-11.)) #m**3 kg**(-1) s**(-2)
AU_U       = AU_SI/R_sun_SI                             #from dist AU to code units (U)
kmsec_U    = 1000./np.sqrt(G_new_SI*M_sun_SI/R_sun_SI)  #from vel km/sec to code units (U)
time_U     = np.sqrt((R_sun_SI**3)/(G_new_SI*M_sun_SI)) #from time sec to code units(U)
Rsch_1Msun_unitRsun = ((2.*G_new_SI*(1.*M_sun_SI))/(c_SI**2.))/R_sun_SI

In [23]:

#bin_f = 2.*np.arctan((((1.+ecc_bin)/(1.-ecc_bin))**(1./2.))*np.tan(sol_eccE/2.))
bin_f = 2.*np.arctan(((1+ecc_bin)/(1-ecc_bin))**(1./2))*np.tan(ecc_anom/2.)
#Redefinition of mass variables
mb1 = 2#b1_mass
mb2 = 1#b2_mass

#SMA_arr = [0.0279028]
#SMA_AU_arr          = np.array([SMA_arr[0]]) #[1e-3]     #input in AU of the binary
#MCvalues_SMA_arr    = AU_U*np.array(SMA_AU_arr)

orbd = 6

#calc/define:
Mb12_SI     = (mb1+mb2)*M_sun_SI
pfac_SI     = ((orbd*R_sun_SI)*(1.-ecc_bin**2))
hfac_SI     = (np.sqrt(G_new_SI*Mb12_SI*pfac_SI))
rdist_SI    = (pfac_SI/(1.+ecc_bin*np.cos(bin_f)))
v_ang_SI    = (hfac_SI/rdist_SI)
v_rad_SI    = ((ecc_bin*np.sin(bin_f)/pfac_SI)*hfac_SI)
#calc pos and vel of reduced-mass object:
posx_U  = - ((rdist_SI*np.cos(bin_f))/R_sun_SI)
posy_U  =   ((rdist_SI*np.sin(bin_f))/R_sun_SI)
velx_U  = - ((v_ang_SI*np.sin(bin_f) - v_rad_SI*np.cos(bin_f))*(kmsec_U/1000.))
vely_U  = - ((v_rad_SI*np.sin(bin_f) + v_ang_SI*np.cos(bin_f))*(kmsec_U/1000.))
#define pos/vel vecs:
posvec_U = np.array([posx_U, posy_U, 0.0])
velvec_U = np.array([velx_U, vely_U, 0.0])

#change to binary COM:
b1_posxyz_binCM    = - (mb2/(mb1+mb2))*posvec_U
b1_velxyz_binCM    = - (mb2/(mb1+mb2))*velvec_U
b2_posxyz_binCM    =   (mb1/(mb1+mb2))*posvec_U
b2_velxyz_binCM    =   (mb1/(mb1+mb2))*velvec_U

In [24]:
print(b1_posxyz_binCM)

[-0.00145504 -0.00525382 -0.        ]


In [18]:
MCvalues_SMA_arr

array([5.99913693])